# UnitTest
전체 테스트 하는것이 아닌, 각 함수별로 test함수를 만들어 각각 실행하는 방법

In [1]:
import numpy as np

class Variable:
    def __init__(self, data) -> None:
        self.data = data
        self.grad = None
        self.creator = None
        
    # creator 설정 함수
    def set_creator(self, func):
        self.creator = func
        
    # 역전파 함수
    # 재귀적으로 구현
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        f = self.creator
        if f is not None:
            x = f.input_
            x.grad = f.backward(self.grad)
            x.backward()
            
    # 역전파 함수
    # 재귀적으로 구현
    # def backward(self):
    #     if self.grad is None:
    #         self.grad = np.ones_like(self.data)
            
    #     f = self.creator
    #     if f is not None:
    #         x = f.input_
    #         x.grad = f.backward(self.grad)
    #         x.backward()
           
    # 역전파 함수를 한번에 하기 위한 함수 
    # 재귀로 하는것보다 while문을 사용하여 반복문으로 구현하는게 더 효율적
    def backward_list(self):
        # 요부분 없으면 에러남
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input_, f.output_
            x.grad = f.backward(y.grad)
            if x.creator is not None:
                funcs.append(x.creator)
    
class Function:
    def __call__(self, input_):
        x = input_.data
        y = self.forward(x)
        output_ = Variable(y)
        output_.set_creator(self)
        self.input_ = input_
        self.output_ = output_
        return output_
    
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, dy):
        raise NotImplementedError() 
    
    
class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self, dy):
        x = self.input_.data
        dx = 2 * x * dy
        return dx

class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, dy):
        x = self.input_.data
        dx = np.exp(x) * dy
        return dx

def exp(x):
    return Exp()(x)

def square(x):
    return Square()(x)

In [2]:
import unittest

# unittest.TestCase를 상속받아 테스트 클래스를 만든다.
class SquareTest(unittest.TestCase):
    # 테스트 함수는 test로 시작하는 암묵적인 규칙
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        
        # y.data 하고 np.array(4.0)과 같은지 비교
        self.assertEqual(y.data, np.array(4.0))
        
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward_list()
        self.assertEqual(x.grad, np.array(6.0))

# 테스트를 실행하는 코드
# terminal에서 python -m unittest {파일이름} 으로 실행해야함
# unittest.main() 사용하면 jupyter에서도 unittest 바로 실행가능함
unittest.main(argv=[''], verbosity=2, exit=False)

test_backward (__main__.SquareTest.test_backward) ... ok
test_forward (__main__.SquareTest.test_forward) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


In [3]:
import unittest

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        self.assertEqual(y.data, np.array(4.0))
        
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        self.assertEqual(x.grad, np.array(6.0))
        
unittest.main(argv=[''], verbosity=2, exit=False)

test_backward (__main__.SquareTest.test_backward) ... ok
test_forward (__main__.SquareTest.test_forward) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


unittest 실행 도중 오류가 계속 발생했었는데<br><br>

backward는 재귀적으로, backward_list는 반복문으로 짠 역전파 함수인데 단일 함수일 때, <br>
backward_list 대신에 backward를 하면 self.grad가 존재하지 않아서 실패가 뜬다.<br>

```python
if self.grad is None:
    self.grad = np.ones_like(self.data)
```

이 문구를 추가를 안해서 self.grad가 존재하지 않았던것.

따라서 unittest뿐만 아니라 backward를 할 때 오류가 발생한다.

---

# 가변 인수 받기
입력값이 가변적인 인수가 들어올 때 처리(EX 입력값이 여러개가 올 수 있고, 출력값이 여러개일수도 있다.)
+ 입력값을 튜플로 변환하고 튜플로 반환 -> 모든 값을 *이용하여 unpack하여 각각의 인수로 변경
+ 반환값도 변수로 변경
+ ### 즉 모든 입출력값을 단일 변수로 변환한 뒤 함수를 실행한다.

In [4]:
class Function:
    def __call__(self, inputs):
        # inputs값이 여러개일 수 있기 때문에 받은 inputs를 리스트로 저장
        x_s = [x.data for x in inputs]
        y_s = self.forward(x_s)
        
        # forward한 뒤 각각 출력값을 outputs 리스트에 저장
        outputs = [Variable(as_array(y)) for y in y_s]
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs
    
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, dy):
        raise NotImplementedError()

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

class Add(Function):
    def forward(self, x_s):
        x0, x1 = x_s
        y = x0 + x1
        return (y, )

def add(x_s):
    return Add()(x_s)

In [5]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add([x0, x1])
y[0].data

array(5)

# 모든 class 가변값에 대하여 대응 & 코드 개선

In [6]:
import numpy as np

class Variable:
    def __init__(self, data) -> None:
        self.data = data
        self.grad = None
        self.creator = None
        
        # 역전파시 함수 순서 중요하기 때문에 generation을 추가하여 함수 순서를 기억하도록 함
        self.generation = 0
        
    def set_creator(self, func):
        self.creator = func
        # 역전파시 함수 순서 중요하기 때문에 generation을 추가하여 함수 순서를 기억하도록 함
        self.generation = func.generation + 1
        
    def clear_grad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        
        # 세대별로 정렬하기 위해서 리스트에 담아 역전파 함수를 구현
        funcs = []
        seen_set = set()
        
        # 함수를 리스트에 넣기
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
                
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop()
            
            # 각 출력값의 미분값을 리스트로 저장
            dy_s = [output.grad for output in f.outputs]
            # unpack을 하여 역전파를 실행
            dx_s = f.backward(*dy_s)
            
            # dx_s가 tuple이 아닌 경우 tuple로 변환
            if not isinstance(dx_s, tuple):
                dx_s = (dx_s,)
            
            # 각 입력값에 미분값을 저장
            for x, dx in zip(f.inputs, dx_s):
                # 처음 값은 미분값을 그대로 저장, 이후 값은 기존 미분값에 추가하도록 함
                if x.grad is None:
                    x.grad = dx
                else:
                    x.grad = x.grad + dx
            
                if x.creator is not None:
                    add_func(x.creator)
        
    
class Function:
    # inputs를 *arg로 받도록 수정 -> inputs가 여러개일 수 있기 때문에 1개든 여러개든 tuple로 받을 수 있도록 함
    def __call__(self, *inputs):
        # inputs의 각 원소의 data를 리스트로 저장
        x_s = [x.data for x in inputs]
        
        # unpack하여 forward 함수 실행
        # function를 override할 때 규칙을 일정하게 하기 위해 unpack
        y_s = self.forward(*x_s)
        
        # y_s가 tuple이 아닌 경우 tuple로 변환
        if not isinstance(y_s, tuple):
            y_s = (y_s, )
        
        def as_array(x):
            if np.isscalar(x):
                return np.array(x)
            return x
    
        # 각 출력값을 Variable로 변환
        outputs = [Variable(as_array(y)) for y in y_s]
        
        # generation중 큰값을 선택하여 generation을 설정
        self.generation = max([x.generation for x in inputs])
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        
        # output이 1개인 경우 첫번째 원소만 반환(하나밖에 없기 때문), 아니면 전체 반환
        # 근데 꼭 이렇게 해야하나? -> 그냥 outputs를 반환하면 되지 않을까?
        # 공부하다보면 출력값 하나여도 대부분 리스트로 반환하는 것 같은데?
        # return outputs
        # 근데 지금 모든 함수 출력은 단일값이기 때문에 outputs으로만 반환(list로 반환)하면 매 function출력마다 [0] 이런식으로 써야함
        # 이거 수정해야하는데 지금 function은 출력값이 단일값이므로 나중에 수정
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, dy):
        raise NotImplementedError()


# 모든 function class
# add면 x_s가 아닌 x0, x1을 받도록 고정
class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self, dy):
        # function class 변경에 맞춰 수정
        x = self.inputs[0].data
        dx = 2 * x * dy
        return dx

class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, dy):
        # function class 변경에 맞춰 수정
        x = self.inputs[0].data
        dx = np.exp(x) * dy
        return dx
    
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, dy):
        return dy, dy

def exp(x):
    return Exp()(x)

def square(x):
    return Square()(x)

def add(x0, x1):
    return Add()(x0, x1)

# 일반적인 상황에서 정상 작동

In [7]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(f'z: {z.data}')
print(f'x: {x.grad}, y: {y.grad}')

z: 13.0
x: 4.0, y: 6.0


# input값이 같은 값이 들어올 때
+ 기대값 : x.grad == 2이여야함
+ 실제값 : x.grad == 1
+ 따라서 처음 값은 미분값을 그대로 저장, 이후 값은 기존 미분값에 추가하도록 함

In [8]:
tmp = add(x, x)
print(f'y = {tmp.data}')

tmp.backward()
print(f'x.grad = {x.grad}')

y = 4.0
x.grad = 6.0


# 한 변수를 두번 이상 사용할 때
+ 기대값: 첫번째, 두번째 backward값 동일
+ 실제값: 첫번째, 두번째 backward값 다름
+ 따라서 gradients를 초기화하는 함수를 variable class에 따로 제작

In [9]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(f'1st x.grad = {x.grad}')

# x.grad 초기화
x.clear_grad()
y = add(x, x)
y.backward()
print(f'2nd x.grad = {x.grad}')

1st x.grad = 2.0
2nd x.grad = 2.0


In [10]:
x = Variable(np.array(3.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(f'y.data = {y.data}')
print(f'x.grad = {x.grad}')

y.data = 162.0
x.grad = 216.0


---

# 메모리 누수 방지를 위한 약한 참조, 역전파 개선
+ 처리 속도와 메모리 사용량의 개선이 필요함
+ 호출한 함수를 사용하고 난 뒤 메모리 해제를 해야함
+ 함수가 서로 참조를 하여 순환 참조가 되면 메모리 삭제가 힘듬 -> 약한 참조를 이용하여 참조한 후 참조를 제거하면 인스턴스가 제가되게 해야함

In [11]:
import weakref
import numpy as np
import contextlib

class Config:
    enable_backprop = True

# 역전파 활성/비활성을 설정하는 함수
@contextlib.contextmanager
def using_config(name, value):
    # with문 시작시 현재 설정값을 저장하고 새로운 설정값으로 변경
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        # finally는 with문이 끝나면 무조건 실행되는 부분
        # 설정값을 원래대로 변경
        setattr(Config, name, old_value)

# 기울기가 필요 없는 경우에 호출
def no_grad():
    return using_config('enable_backprop', False)

class Variable:
    def __init__(self, data) -> None:
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1
        
    def clear_grad(self):
        self.grad = None
        
    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
                
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop()
            
            # output()으로 변경하여 약한 참조로 접근하도록 변경
            dy_s = [output().grad for output in f.outputs]
            dx_s = f.backward(*dy_s)
            
            if not isinstance(dx_s, tuple):
                dx_s = (dx_s,)
            
            for x, dx in zip(f.inputs, dx_s):
                if x.grad is None:
                    x.grad = dx
                else:
                    x.grad = x.grad + dx
            
                if x.creator is not None:
                    add_func(x.creator)
                
            # 불필요한 미분값 제거
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None
    
    
class Function:
    def __call__(self, *inputs):
        x_s = [x.data for x in inputs]
        y_s = self.forward(*x_s)
        
        if not isinstance(y_s, tuple):
            y_s = (y_s,)
        
        def as_array(x):
            if np.isscalar(x):
                return np.array(x)
            return x
    
        outputs = [Variable(as_array(y)) for y in y_s]
    
        # 역전파가 필요한 경우에만 진행
        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            # self.outputs를 약한 참조를 하게 만들어서 이후 역전파시 약한 참조를 통해 접근 가능
            self.outputs = [weakref.ref(output) for output in outputs]
        
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, dy):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self, dy):
        x = self.inputs[0].data
        dx = 2 * x * dy
        return dx

class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, dy):
        x = self.inputs[0].data
        dx = np.exp(x) * dy
        return dx
    
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, dy):
        return dy, dy

def exp(x):
    return Exp()(x)

def square(x):
    return Square()(x)

def add(x0, x1):
    return Add()(x0, x1)

In [12]:
for i in range(10):
    x = Variable(np.random.randn(10000))
    y = square(square(square(x)))
    y.backward()
    print(y.grad)
    
# y.grad가 [[1,1,1,1,,], [1,1,1,1,,], ...]로 출력되야하는데 retain_grad 때문에 none으로 출력됨

None
None
None
None
None
None
None
None
None
None


In [13]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

# retain_grad가 없다면 y.grad, t.grad가 출력됨
# 중간과정의 미분값을 제거하여 메모리를 절약할 수 있음

None None
2.0 1.0


In [14]:
Config.enable_backprop = True
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))
y.backward()
print(y.grad)

None


In [15]:
Config.enable_backprop = False
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))
print(y)

In [16]:
# no_grad()를 사용하여 역전파를 하지 않도록 설정

with no_grad():
    x = Variable(np.ones((100, 100, 100)))
    y = square(square(square(x)))
    print(y)
    

# contextlib.contextmanager 말고
# class로 만들어서
# __enter__와 __exit__을 구현하면
# with문을 동일한 기능으로 사용할 수 있음

# class config_test:
#     def __enter__(self):
#         print('start')
#         return self
#     def __exit__(self, exc_type, exc_value, traceback):
#         print('end')
        
# with config_test() as c:
#     print('process')
    
# 이렇게하면
# start
# process
# end
# 로 출력됨

# Variable class 편의성 개선
+ 변수 이름 추가
+ property decorator를 사용하여 numpy의 일부 함수들을 똑같이 쓸 수 있도록 추가
+ len, print를 할 때 적절한 값이 나오도록 추가

In [17]:
import weakref
import numpy as np
import contextlib

class Config:
    enable_backprop = True

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

def no_grad():
    return using_config('enable_backprop', False)

class Variable:
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은 지원하지 않습니다.')
            
        self.data = data
        # 변수 이름 추가
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1
        
    def clear_grad(self):
        self.grad = None
        
    # property를 사용하여 shape, ndim, size, dtype 호출 시 해당 함수가 호출되도록 함(변수가 numpy이면 numpy의 함수를 호출하도록 함)
    @property
    def shape(self):
        return self.data.shape
    @property
    def ndim(self):
        return self.data.ndim
    @property
    def size(self):
        return self.data.size
    @property
    def dtype(self):
        return self.data.dtype
        
        
    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
                
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
                
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop()
            
            dy_s = [output().grad for output in f.outputs]
            dx_s = f.backward(*dy_s)
            
            if not isinstance(dx_s, tuple):
                dx_s = (dx_s, )
            
            for x, dx in zip(f.inputs, dx_s):
                if x.grad is None:
                    x.grad = dx
                else:
                    x.grad = x.grad + dx
            
                # 이부분 지금처럼 for문 안에 넣어줘야함
                if x.creator is not None:
                    add_func(x.creator)
                
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None
                    
                    
    # __len__을 추가하여 len함수로 호출 시 data의 길이를 반환하도록 함
    def __len__(self):
        return len(self.data)
    # __repr__을 추가하여 print로 호출 시 출력되는 문자열을 지정
    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return f'variable({p})'
    
    
class Function:
    def __call__(self, *inputs):
        x_s = [x.data for x in inputs]
        y_s = self.forward(*x_s)
        
        if not isinstance(y_s, tuple):
            y_s = (y_s,)
        
        def as_array(x):
            if np.isscalar(x):
                return np.array(x)
            return x
    
        outputs = [Variable(as_array(y)) for y in y_s]
    
        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]
        
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, dy):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self, dy):
        x = self.inputs[0].data
        dx = 2 * x * dy
        return dx

class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, dy):
        x = self.inputs[0].data
        dx = np.exp(x) * dy
        return dx
    
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, dy):
        return dy, dy

def exp(x):
    return Exp()(x)

def square(x):
    return Square()(x)

def add(x0, x1):
    return Add()(x0, x1)

In [18]:
x = Variable(np.array([[1,2,3], [4,5,66]]), 'x')
print(x.shape)
print(x.name)

(2, 3)
x


In [19]:
print(x)
print(len(x))

variable([[ 1  2  3]
          [ 4  5 66]])
2


# 곱셈 연산자 추가

In [20]:
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y
    
    def backward(self, dy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return x1 * dy, x0 * dy
    
def mul(x0, x1):
    return Mul()(x0, x1)

In [21]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = add(mul(a, b), c)
y.backward()

print(y)
print(b.grad)
print(a.grad)

# if x.creator is not None:
#     add_func(x.creator)
# 이부분 for문 안에 넣어줘야함

variable(7.0)
3.0
2.0


# 연산자 관리

In [22]:
import weakref
import numpy as np
import contextlib

class Config:
    enable_backprop = True

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

def no_grad():
    return using_config('enable_backprop', False)

# obj를 Variable로 변환하는 함수
def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

# obj를 numpy로 변환하는 함수
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

class Variable:
    # __array_priority__를 설정하여 array와 우선순위를 정함
    # np.array([2.0]) + Variable(np.array([3.0]))을 실행하면
    # np.array([2.0])가 먼저 실행되고 __add__가 실행되는데
    # __array_priority__가 더 높은 Variable이 먼저 실행되도록 함
    __array_priority__ = 200
    
    def __init__(self, data, name=None):
        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1
        
    def clear_grad(self):
        self.grad = None
        
    @property
    def shape(self):
        return self.data.shape
    
    @property
    def ndim(self):
        return self.data.ndim
    
    @property
    def size(self):
        return self.data.size
    
    @property
    def dtype(self):
        return self.data.dtype
        
    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
                
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop()
            
            dy_s = [output().grad for output in f.outputs]
            dx_s = f.backward(*dy_s)
            
            if not isinstance(dx_s, tuple):
                dx_s = (dx_s,)
            
            for x, dx in zip(f.inputs, dx_s):
                if x.grad is None:
                    x.grad = dx
                else:
                    x.grad = x.grad + dx
            
                if x.creator is not None:
                    add_func(x.creator)
                    
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None
                    
    def __len__(self):
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return f'variable({p})'
    
    # __mul__을 추가하면 mul(x0, x1) 대신 x0 * x1로 사용 가능함
    # 나머지도 동일하게 설정
    def __mul__(self, other):
        return mul(self, other)
    def __add__(self, other):
        return add(self, other)
    
    # __rmul__을 추가하면 mul(x0, x1) 대신 x1 * x0로 사용 가능함
    # 나머지도 동일하게 설정
    def __rmul__(self, other):
        return mul(other, self)
    def __radd__(self, other):
        return add(other, self)
    
class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]
        
        x_s = [x.data for x in inputs]
        y_s = self.forward(*x_s)
        
        if not isinstance(y_s, tuple):
            y_s = (y_s,)
        
        def as_array(x):
            if np.isscalar(x):
                return np.array(x)
            return x
    
        outputs = [Variable(as_array(y)) for y in y_s]
    
        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]
        
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, dy):
        raise NotImplementedError()



class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self, dy):
        x = self.inputs[0].data
        dx = 2 * x * dy
        return dx

class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, dy):
        x = self.inputs[0].data
        dx = np.exp(x) * dy
        return dx
    
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, dy):
        return dy, dy
    
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y
    
    def backward(self, dy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return x1 * dy, x0 * dy
    
# 나머지 계산 부호들도 추가
class Neg(Function):
    def forward(self, x):
        return -x
    
    def backward(self, dy):
        return -dy
class Sub(Function):
    def forward(self, x0, x1):
        return x0 - x1
    
    def backward(self, dy):
        return dy, -dy
class Div(Function):
    def forward(self, x0, x1):
        return x0 / x1
    
    def backward(self, dy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return dy / x1, -x0 * dy / x1 ** 2
class Pow(Function):
    def __init__(self, c):
        self.c = c
        
    def forward(self, x):
        return x ** self.c
    
    def backward(self, dy):
        x = self.inputs[0].data
        c = self.c
        dx = c * x ** (c - 1) * dy
        return dx
        
    
def exp(x):
    return Exp()(x)

def square(x):
    return Square()(x)

def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)

def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)

# 나머지 계산 부호들도 추가
def neg(x):
    return Neg()(x)
def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)
def rsub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x1, x0)
def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)
def rdiv(x0, x1):
    x1 = as_array(x1)
    return Div()(x1, x0)
def pow(x, c):
    return Pow(c)(x)


Variable.__mul__ = mul
Variable.__add__ = add
Variable.__rmul__ = mul
Variable.__radd__ = add
Variable.__neg__ = neg
Variable.__sub__ = sub
Variable.__rsub__ = rsub
Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv
Variable.__pow__ = pow

In [23]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))

y = a * b
y.backward()

print(y)

variable(6.0)


In [24]:
# variable * np.array는 연산 불가
# 마찬가지로 1 + variable도 연산 불가
a = Variable(np.array(3.0))

y = a * np.array(2.0)
y.backward()

print(y)

variable(6.0)


In [25]:
a = Variable(np.array(3.0))

y = a + np.array(2.0)
y.backward()

print(y)

variable(5.0)


In [26]:
# mul 좌우 바꿔도 가능하게 함
a = Variable(np.array(3.0))

y = np.array(2.0) * a
y.backward()

print(y)

# add 좌우 바꿔도 가능하게 함
a = Variable(np.array(3.0))

y = np.array(2.0) + a
y.backward()

print(y)

variable(6.0)
variable(5.0)


In [27]:
a = Variable(np.array([3.0]))

y = np.array([2.0]) + a
y.backward()

print(y)

variable([5.])


In [28]:
# 나머지 계산 부호들도 추가
x = Variable(np.array(2.0))
y = -x
y.backward()
print(y)

x = Variable(np.array(2.0))
y = x - 3.0
y.backward()
print(y)

x = Variable(np.array(2.0))
y = 5.0 - x
y.backward()
print(y)

x = Variable(np.array(2.0))
y = x / 3.0
y.backward()
print(y)

x = Variable(np.array(2.0))
y = 5.0 / x
y.backward()
print(y)

x = Variable(np.array(2.0))
y = x ** 3
y.backward()
print(y)

variable(-2.0)
variable(-1.0)
variable(3.0)
variable(0.6666666666666666)
variable(2.5)
variable(8.0)


---

+ 궁금한 사항은 중간중간 코드로 해결
+ 책에선 나오지 않는 내용이지만
    ```python
    class Function:
        def __call__(self, *inputs):
            
            return outputs if len(outputs) > 1 else outputs[0]
    ```
    outputs로 통일해도 되지 않나?
    그냥 반환값이 하나만 있는 경우 편의성을 위해서 outputs[0]을 하는건데 함수의 통일성으로 하려면 outputs만 하는게 좋지 않나